# Replace humans with Stick figures

## Imports and constants

In [7]:
import cv2, time, pandas
from datetime import datetime
import os
import glob
import numpy as np

DIFF_THRESHOLD = 30
SIZE = (640, 480)

In [8]:
STATIC_BACKGROUND = None

In [9]:
VIDEO_PATH = "input/parent-falling.mp4"
IMG_PATH = "res/stick.png"

SyntaxError: invalid syntax (<ipython-input-9-f5f3213c1e0c>, line 2)

## Video captue

Use cv2 `HOGDescriptor_getDefaultPeopleDetector` to detect human bounding box. Use non-maxima suppression to combine overlapping boxes

TODO: better human detection can be used to account for other moving object

### Non-maxima suppression

In [5]:
# import the necessary packages
import numpy as np
# Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh=0.3):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    # initialize the list of picked indexes	
    pick = []
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int32")

### Get background

In [6]:
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = SIZE
background_sum = np.zeros((height,width, 3), np.int32)
background_cnt = np.zeros((height,width, 3), np.int32)
prev_img = None

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

stick = imread("")

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.resize(frame, SIZE)
    frame = cv2.GaussianBlur(frame, (7, 7), 0)

    # Our operations on the frame come here
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # compute difference
#     diff_frame = cv2.absdiff(prev_img, gray)
#     thresh_frame = cv2.threshold(diff_frame, DIFF_THRESHOLD, 255, cv2.THRESH_BINARY)[1]
#     thresh_frame = cv2.dilate(thresh_frame, None, iterations=2)
#     prev_img = gray
    
    addition = frame
    
    boxes, weights = hog.detectMultiScale(frame, winStride=(8,8) )
    boxes, weights = np.asarray(boxes), np.asarray(weights)
    boxes = boxes[weights.flatten() > 0.5]
    boxes = non_max_suppression_fast(boxes, weights)
    
#     cnts, _ = cv2.findContours(thresh_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for box in boxes:
        (x, y, w, h) = box
        addition[y:y+h, x:x+w, :] = 0
#         cv2.rectangle(addition, (x, y), (x + w, y + h), (0, 255, 0), 3)
        
    
#     for contour in cnts:
#         if cv2.contourArea(contour) < 100:
#             continue
#         (x, y, w, h) = cv2.boundingRect(contour)
#         addition[x:x+w, y:y+h, :] = 0
#         cv2.rectangle(addition, (x, y), (x + w, y + h), (0, 255, 0), 3)
    
    cv2.imshow('frame',addition)
    cv2.waitKey(1)
    background_sum += addition
    background_cnt += 1 * (addition > 0)
    display_cnt = background_cnt
    display_cnt[display_cnt == 0] = 1 
    display_frame = background_sum // display_cnt
    cv2.imwrite('color_img.jpg', display_frame)
#     print(display_cnt[10, 10, 1],background_sum[10, 10, 1], frame[10, 10, 1], display_frame[10, 10, 1])
    img = cv2.imread('color_img.jpg', 0)
    cv2.imshow('back',img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

1 104 104 104
2 208 104 104
3 312 104 104
4 416 104 104
5 520 104 104
6 624 104 104
7 728 104 104
8 832 104 104
9 936 104 104
10 1040 104 104
11 1144 104 104
12 1248 104 104
13 1352 104 104
14 1456 104 104
15 1560 104 104
16 1664 104 104
17 1768 104 104
18 1872 104 104
19 1976 104 104
20 2080 104 104
21 2184 104 104
22 2288 104 104
23 2392 104 104
24 2496 104 104
25 2600 104 104
26 2704 104 104
27 2808 104 104
28 2912 104 104
29 3016 104 104
30 3120 104 104
31 3225 105 104
32 3330 105 104
33 3435 105 104
34 3540 105 104
35 3645 105 104
36 3750 105 104
37 3855 105 104
38 3960 105 104
39 4065 105 104
40 4170 105 104
41 4275 105 104
42 4380 105 104
43 4485 105 104
44 4590 105 104
45 4695 105 104
46 4800 105 104
47 4905 105 104
48 5010 105 104
49 5115 105 104
50 5220 105 104
51 5325 105 104
52 5430 105 104
53 5535 105 104
54 5640 105 104
55 5745 105 104
56 5850 105 104
57 5955 105 104
58 6060 105 104
59 6165 105 104
60 6270 105 104
61 6374 104 104
62 6478 104 104
63 6582 104 104
64 6686 10